In [10]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output

# Import your CRUD module
from animal_shelter import AnimalShelter

# Configure the plotting routines
import numpy as np
import pandas as pd

# Data Manipulation / Model
username = "aacuser"
password = "SNHU1234"
host = 'nv-desktop-services.apporto.com'
port = 30351
database = "AAC"
collection = "animals"

shelter = AnimalShelter(username, password, host, port, database, collection)

# Fetch data from MongoDB and prepare the dataframe
df = pd.DataFrame.from_records(shelter.read({}))

if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Dashboard Layout / View
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Dylan Cavazos SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Button('Show Cats', id='submit-button-one', n_clicks=0),
    html.Button('Show Dogs', id='submit-button-two', n_clicks=0),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",  # Enabling single-row selection
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],  # Select the first row by default
        page_action="native",
        page_current=0,
        page_size=10
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        id='map-id',
        className='col s12 m6',
    )
])

#############################################
# Interaction Between Components / Controller
#############################################

# This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(Output('datatable-id', "data"),
              [Input('submit-button-one', 'n_clicks'),
               Input('submit-button-two', 'n_clicks')
              ])

def on_click(button1, button2):
    #start case
    df = pd.DataFrame.from_records(shelter.read({}))
    
    # use higher number of button clicks to determine filter type.
    if button1 > 0 and button1 >= button2:
        df = pd.DataFrame.from_records(shelter.read({"animal_type" : "Cat"}))
    elif button2 > 1 and button2 > button1:
        df = pd.DataFrame.from_records(shelter.read({"animal_type" : "Dog"}))
        
    # Cleanup Mongo_id field
    if '_id' in df.columns:
        df.drop(columns=['_id'],inplace=True)
        
    return df.to_dict('records')

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)

def update_map(viewData, index):    
    
    if not viewData:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None or len(index) == 0:
        row = 0
    else: 
        row = index[0]

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               
           center=[30.75,-97.48], zoom=10, children=[
           dl.TileLayer(id="base-layer-id"),
               
           # Marker with tool tip and popup
           # Column 13 and 14 define the grid-coordinates for 
           # the map
           # Column 4 defines the breed for the animal
           # Column 9 defines the name of the animal
               
           dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
              children=[
              dl.Tooltip(dff.iloc[row,4]),
              dl.Popup([
                 html.H1("Animal Name"),
                html.P(dff.iloc[row,9])
             ])
          ])
       ])
    ]

# Run app and display result inline in the notebook
app.run_server(debug='inline', port=8051)

Connection to MongoDB successful.
Dash app running on http://127.0.0.1:8051/
